In [ ]:
# Linking with google drive
# simple upload to github, manual here: https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/My Drive/time_series_analysis/time_series_prediction

/content/drive/My Drive/time_series_analysis/time_series_prediction


In [2]:
#!git clone https://{git_token}@github.com/{username}/{repository}

In [3]:
# trying to import python module as a separate script

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/dataprocessing/')
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/models/')

In [5]:
import datapreparation
#import baselines
from matplotlib import pyplot as plt

In [6]:
%load utils/plotting.ipynb

In [7]:
%load models/baselines.ipynb

In [7]:
#%load time_series_prediction/models/baselines.py

In [8]:
#%load time_series_prediction/dataprocessing/datapreparation.py

In [8]:
import pandas as pd

In [9]:
sales_data = pd.read_csv("/content/sales.csv",   header = 0, sep = ";")
categories = pd.read_csv("/content/categories.csv",   header = 0, sep = ";")
traffic = pd.read_csv("/content/traffic.csv",   header = 0, sep = ";")

In [10]:
traffic['DateTime'] = traffic['week_starting_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
traffic.drop("week_starting_date", axis = 1, inplace = True)

traffic = traffic.set_index(["product_id", "DateTime"])
categories = categories.set_index(["product_id"])

In [11]:
# extended only to global max date horizon
# check nulls here because looks strange
sales_df_combined_right = datapreparation.prepare_sales_data(sales_data, categories, traffic, True, True, [1,2,3])

In [13]:
sales_df_combined_right

sales  category_id  ...  sales_lag_3W  traffic_lag_3W
product_id DateTime                        ...                              
1          2020-07-06    1.0           10  ...           0.0             0.0
           2020-07-13    0.0           10  ...           0.0             0.0
           2020-07-20    1.0           10  ...           0.0             0.0
           2020-07-27    3.0           10  ...           1.0             NaN
           2020-08-03    4.0           10  ...           0.0             NaN
...                      ...          ...  ...           ...             ...
3135       2020-11-30  102.0           43  ...          21.0           100.0
           2020-12-07   45.0           43  ...          29.0            81.0
           2020-12-14    8.0           43  ...         124.0           108.0
           2020-12-21   17.0           43  ...         102.0           142.0
           2020-12-28    7.0           43  ...          45.0            90.0

[217439 rows x 12 columns]

In [12]:
sample = sales_df_combined_right.loc[[2658, 1308]]

In [ ]:
# for i in all_items:
#   print("*******************")
#   print(i)
#   tmp_dataset = sales_df_combined_right.loc[i]
#   #n_splits_tmp = int(np.floor((tmp_dataset.shape[0] - 3)/2))
#   try:
#     tscv = TimeSeriesSplit(n_splits=5, test_size = 3)
#     for tr_index, val_index in tscv.split(tmp_dataset):
#       print (tr_index, val_index)
#       X_tr, X_val = tmp_dataset[["sales"]].iloc[tr_index], tmp_dataset[["sales"]].iloc[val_index]
#       print(X_tr)
#       print(X_val)
#       forecast = get_naive_forecast(X_tr.sales, horizon = 3)
#       print(forecast)
#       print(mean_absolute_percentage_error(X_val.sales, forecast))
#   except Exception as e:
#     print(e)

In [ ]:
# securing proper cross validation without errors

In [ ]:
# linear regression

In [ ]:
# hierarchical clustering

### Git commands

In [16]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Master_exploration.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	utils/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add .

In [ ]:
!git config --global user.email "katarzyna.staniewicz0504@gmail.com"

In [ ]:
!git commit -m 'Updates'

[master 009964d] Updates
 3 files changed, 3 insertions(+), 2 deletions(-)
 rewrite Master_exploration.ipynb (62%)
 create mode 100644 models/baselines.ipynb
 rewrite tests/baselines_tests.ipynb (98%)


In [ ]:
!git push

Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (7/7), 2.75 KiB | 469.00 KiB/s, done.
Total 7 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/katarzynasornat/time_series_prediction
   a20a2ee..009964d  master -> master


In [15]:
model_drift = NaiveDrift(horizon=3)
model_naive = baselines.NaiveForecast(horizon=3)

NameError: ignored

In [ ]:
drift_forecast = model_drift.fit(sales_df_combined_right.loc[2658].sales[:-3]).predict(sales_df_combined_right.loc[2658].sales[-3:])
naive_forecast = model_naive.fit(sales_df_combined_right.loc[2658].sales[:-3]).predict(sales_df_combined_right.loc[2658].sales[-3:])

In [ ]:
# plotting functions 

In [ ]:
forecasts = {'drift_foreast': drift_forecast, 'naive_forecast': naive_forecast}

In [ ]:
plot_original_and_forecasts(original_series = sales_df_combined_right.loc[2658].sales, dict_with_forecasts = forecasts)

In [ ]:
class TimeSeries(object):
  def __init__(self, series_with_numbers):
    self._time_col = time_col
    self._values = df
    self._product_id = df.product_id.iloc[0]

    self._time_indexed = pd.to_datetime(df[time_col])

    self.init_ts()
    self._how_many_nonzero_initaly()
  
  def init_ts(self):
    self._values.index = self._time_indexed
    self._ts = self._values.asfreq('W-Mon').sort_index(ascending = True).fillna(0)

  def _how_many_nonzero_initaly(self):
    self._non_zero_weeks = self._values.shape[0]
    self._non_zero_share = 100*round(self._non_zero_weeks/self._ts.shape[0],4)

  def plot_time_series(self, col_to_plot):
    self._ts[[col_to_plot]].plot(marker='o', title = f"product_id = {self._product_id}")

In [ ]:
# prod 5 prod 4
ex = TimeSeries("DateTime_x" ,full_sales_dataset.loc[full_sales_dataset['product_id'] == 4])